# AWS Documentation MCP Server on AgentCore Runtime

This notebook demonstrates how to test and deploy the AWS Documentation MCP server to Amazon Bedrock AgentCore Runtime.

## Prerequisites Setup

**Step 1:** Clone the AWS MCP repository
```bash
git clone https://github.com/awslabs/mcp.git
```

**Step 2:** Copy the AWS Documentation MCP server to your project root
```bash
cp -r ./mcp/src/aws-documentation-mcp-server ./
```

**Step 3:** Set up your environment variables in `.env` file:
```
COGNITO_POOL_ID=your_pool_id
COGNITO_REGION=us-east-1
COGNITO_USERNAME=admin
COGNITO_CLIENT_SECRET=your_client_secret
COGNITO_PASSWORD=your_password
AWS_PROFILE=default
AWS_DOCUMENTATION_PARTITION=aws
```

**Step 4:** Follow the instructions below to complete the deployment

## AWS Documentation MCP Server Overview

The AWS Documentation MCP server provides comprehensive access to AWS documentation through 3-4 specialized tools:

### Global AWS Documentation Tools (AWS_DOCUMENTATION_PARTITION=aws)
- **`read_documentation`**: Fetch and convert AWS documentation pages to markdown format
- **`search_documentation`**: Search AWS documentation using the official search API
- **`recommend`**: Get content recommendations for AWS documentation pages

### China AWS Documentation Tools (AWS_DOCUMENTATION_PARTITION=aws-cn)
- **`read_documentation`**: Fetch and convert AWS China documentation pages to markdown format
- **`get_available_services`**: Get a list of available AWS services in China regions

### Key Features
- **Public Documentation Access**: No AWS credentials required for documentation access
- **Markdown Conversion**: Converts HTML documentation to clean markdown format
- **Pagination Support**: Handle long documents with start_index parameter
- **Search Integration**: Access to official AWS documentation search API
- **Recommendation Engine**: Discover related content and recent updates
- **Multi-Region Support**: Support for both global AWS and AWS China documentation
- **Source Citation**: Always provides documentation URLs for reference

### Usage Examples
- "Look up documentation on S3 bucket naming rules. Cite your sources"
- "Search for Lambda function invocation documentation"
- "Recommend content for DynamoDB best practices"
- "Find recent updates for Amazon Bedrock service"

All tools work with public AWS documentation and require no special AWS permissions beyond basic AgentCore runtime access.

## Prerequisites

Before running this notebook, ensure you have:

### System Requirements
- Python 3.10 or higher
- AWS CLI configured with valid credentials
- Docker installed (for containerization)

### AWS Permissions
Your AWS credentials must have permissions for:
- Amazon Bedrock AgentCore
- Amazon ECR (for container registry)
- Amazon Cognito (for authentication)
- IAM (for role creation)
- AWS Systems Manager Parameter Store
- AWS Secrets Manager

**Note**: No special permissions are needed for accessing AWS documentation as it's publicly available.

### Project Structure
- `aws-documentation-mcp-server/` - The MCP server implementation
- `requirements.txt` - Python dependencies
- `utils.py` - Helper functions for Cognito and IAM setup

## 1. Install Dependencies

Install all required Python packages using uv (recommended) or pip:

In [1]:
# Set AWS profile for this session
import os
os.environ['AWS_PROFILE'] = 'ibc2025'
print(f"Using AWS Profile: {os.environ['AWS_PROFILE']}")

Using AWS Profile: ibc2025


In [2]:
# Check current Python and install packages directly
import sys
print(f"Python executable: {sys.executable}")

# Install packages using the current Python interpreter
import subprocess
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    print("✓ All packages installed successfully")
except subprocess.CalledProcessError as e:
    print(f"Error installing packages: {e}")
    
# Verify key packages are available
try:
    import boto3
    print("✓ boto3 available")
except ImportError:
    print("❌ boto3 not available")
    
try:
    from dotenv import load_dotenv
    print("✓ python-dotenv available")
except ImportError:
    print("❌ python-dotenv not available")

Python executable: /opt/homebrew/opt/python@3.11/bin/python3.11
✓ All packages installed successfully
✓ boto3 available
✓ python-dotenv available



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


In [ ]:
!uv pip install -r requirements.txt

## 2. Local Testing

Before deploying to AgentCore Runtime, let's test the AWS Documentation MCP server locally.

### 2.1 MCP Server Wrapper

The `mcp-server.py` file creates a FastMCP wrapper around the AWS Documentation MCP server for AgentCore deployment. The reason we have to do this is because AWS MCP servers are implemented for Stdio local run, we are extending it for remote MCP server using this wrapper:

- Imports tools from the original server implementation based on partition
- Configures the server for HTTP transport on port 8000
- Sets up proper instructions and dependencies
- Enables stateless HTTP mode for AgentCore compatibility
- Supports both global AWS and AWS China documentation partitions

The server exposes these tools:
- **Global AWS**: `read_documentation`, `search_documentation`, `recommend`
- **AWS China**: `read_documentation`, `get_available_services`

In [3]:
%%writefile mcp-server.py
#!/usr/bin/env python3
import os
import sys

# Add the path before any imports
sys.path.insert(0, os.path.abspath("./aws-documentation-mcp-server"))

# Get partition from environment variable
PARTITION = os.getenv('AWS_DOCUMENTATION_PARTITION', 'aws').lower()

# Import the appropriate server based on partition
if PARTITION == 'aws':
    from awslabs.aws_documentation_mcp_server.server_aws import (
        read_documentation, search_documentation, recommend
    )
elif PARTITION == 'aws-cn':
    from awslabs.aws_documentation_mcp_server.server_aws_cn import (
        read_documentation, get_available_services
    )
else:
    raise ValueError(f'Unsupported AWS documentation partition: {PARTITION}.')

# Create a new FastMCP instance with correct parameters
from mcp.server.fastmcp import FastMCP

# Partition-specific instructions
if PARTITION == 'aws':
    instructions = """AWS Documentation MCP Server provides comprehensive tools for accessing global AWS documentation.

    This server enables you to:
    - Read and convert AWS documentation pages to markdown format
    - Search AWS documentation using the official search API
    - Get content recommendations for documentation pages
    - Access recent updates and related content
    - Cite documentation sources for accurate information

    ## Available Tools:
    
    ### read_documentation
    Fetches an AWS documentation page and converts it to markdown format.
    - Supports pagination for long documents using start_index parameter
    - Validates docs.aws.amazon.com URLs ending in .html
    - Preserves document structure, code blocks, and formatting
    
    ### search_documentation
    Searches AWS documentation using the official AWS Documentation Search API.
    - Use specific technical terms rather than general phrases
    - Returns structured results with titles, URLs, and snippets
    - Configurable result limits for focused searches
    
    ### recommend
    Gets content recommendations for an AWS documentation page.
    - Discover related content that might not appear in search results
    - Find newly released information in the "New" section
    - Use as fallback when searches yield insufficient results

    ## Best Practices:
    - For long documents, use multiple read_documentation calls with different start_index values
    - Always cite the documentation URL when providing information to users
    - Use recommend tool to discover related content and recent updates
    - If searches are insufficient, pivot to using recommend for broader discovery

    For more information about AWS Documentation, visit:
    https://docs.aws.amazon.com/
    """
else:  # aws-cn
    instructions = """AWS Documentation MCP Server provides tools for accessing AWS China documentation.

    This server enables you to:
    - Read and convert AWS China documentation pages to markdown format
    - Get a list of available AWS services in China regions
    - Access localized documentation for Chinese AWS regions

    ## Available Tools:
    
    ### read_documentation
    Fetches an AWS China documentation page and converts it to markdown format.
    - Supports pagination for long documents using start_index parameter
    - Works with AWS China documentation URLs
    - Preserves document structure and formatting
    
    ### get_available_services
    Gets a list of available AWS services in China regions.
    - Returns comprehensive service availability information
    - Helps identify which services are available in China regions

    For more information about AWS China, visit:
    https://www.amazonaws.cn/
    """

mcp = FastMCP(
    'awslabs.aws-documentation-mcp-server',
    host="0.0.0.0",
    stateless_http=True,
    instructions=instructions,
    dependencies=[
        'pydantic',
        'httpx',
        'beautifulsoup4',
        'loguru',
    ],
)

# Register tools based on partition
if PARTITION == 'aws':
    mcp.tool(name='read_documentation')(read_documentation)
    mcp.tool(name='search_documentation')(search_documentation)
    mcp.tool(name='recommend')(recommend)
elif PARTITION == 'aws-cn':
    mcp.tool(name='read_documentation')(read_documentation)
    mcp.tool(name='get_available_services')(get_available_services)

if __name__ == "__main__":
    print(f"Starting AWS Documentation MCP server ({PARTITION}) in HTTP mode on http://0.0.0.0:8000")
    mcp.run(transport="streamable-http")

Overwriting mcp-server.py


### 2.2 Local Test Client

The `mcp-client.py` creates a simple test client that:

- Connects to the local MCP server at `http://0.0.0.0:8000/mcp`
- Lists all available tools
- Provides basic connectivity testing

This client helps verify that your MCP server is running correctly before deployment.

In [4]:
%%writefile mcp-client.py
#!/usr/bin/env python3
import asyncio
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def test_server():
    mcp_url = "http://0.0.0.0:8000/mcp"
    
    try:
        async with streamablehttp_client(mcp_url, {}, terminate_on_close=False) as (
            read_stream, write_stream, _
        ):
            async with ClientSession(read_stream, write_stream) as session:
                await session.initialize()
                
                tool_result = await session.list_tools()
                print(f"Found {len(tool_result.tools)} tools:")
                
                for tool in tool_result.tools:
                    print(f"  - {tool.name}")
                
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    asyncio.run(test_server())

Overwriting mcp-client.py


### 2.3 Local Testing Instructions

To test your AWS Documentation MCP server locally:

1. **Terminal 1**: Start the MCP server
   ```bash
   python mcp-server.py
   ```
   Expected output: `Starting AWS Documentation MCP server (aws) in HTTP mode on http://0.0.0.0:8000`
   
2. **Terminal 2**: Run the test client
   ```bash
   python mcp-client.py
   ```
   Expected output: `Found 3 tools:` (for global AWS) or `Found 2 tools:` (for AWS China)

**Note**: Local testing does not require special AWS credentials as AWS documentation is publicly accessible. Only basic internet connectivity is needed.

## 3. Amazon Cognito Authentication Setup

AgentCore Runtime requires JWT-based authentication. We'll use Amazon Cognito to provide bearer tokens for accessing our deployed MCP server.

The `utils.py` file contains helper functions:
- `get_cognito_pool_info()`: Retrieves configuration from an existing Cognito User Pool
- `setup_cognito_user_pool()`: Creates a new User Pool if needed
- `create_agentcore_role()`: Creates the necessary IAM role with proper permissions

In [5]:
import os
from dotenv import load_dotenv
from utils import get_cognito_pool_info, create_agentcore_role

load_dotenv()

pool_id = os.getenv('COGNITO_POOL_ID', 'us-east-1_XXXXX')
region = os.getenv('COGNITO_REGION', 'us-east-1')
    
print(f"Get Cognito user pool info for pool id: {pool_id} in region: {region}")

print("Setting up Amazon Cognito user pool...")
cognito_config = get_cognito_pool_info(pool_id, region)
print("Cognito setup completed ✓")

Get Cognito user pool info for pool id: eu-central-1_PaVtjk8dt in region: eu-central-1
Setting up Amazon Cognito user pool...
Using client ID: 4rit5a00iqft9ak8sl5hb28sr
Making OAuth request to: https://mcp-registry-241533163649-mcp-gateway-registry.auth.eu-central-1.amazoncognito.com/oauth2/token
✅ Successfully obtained bearer token via OAuth client credentials
Pool id: eu-central-1_PaVtjk8dt
Discovery URL: https://cognito-idp.eu-central-1.amazonaws.com/eu-central-1_PaVtjk8dt/.well-known/openid-configuration
Client ID: 4rit5a00iqft9ak8sl5hb28sr
Bearer Token: eyJraWQiOiJYZ1wvblZQWmtZeEtLM3ZicHBPRVQ2cUVYZUZLdE12QkVcLzVyWjJGK3ZoZWs9IiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiI0cml0NWEwMGlxZnQ5YWs4c2w1aGIyOHNyIiwidG9rZW5fdXNlIjoiYWNjZXNzIiwic2NvcGUiOiJtY3AtcmVnaXN0cnlcL3JlYWQgbWNwLXJlZ2lzdHJ5XC93cml0ZSIsImF1dGhfdGltZSI6MTc1NTgwOTQwMiwiaXNzIjoiaHR0cHM6XC9cL2NvZ25pdG8taWRwLmV1LWNlbnRyYWwtMS5hbWF6b25hd3MuY29tXC9ldS1jZW50cmFsLTFfUGFWdGprOGR0IiwiZXhwIjoxNzU1ODEzMDAyLCJpYXQiOjE3NTU4MDk0MDIsInZlcnNpb24iOjIs

## 4. IAM Role Creation

Create an IAM execution role for the AgentCore Runtime with:

### Base Permissions
- Amazon Bedrock model invocation
- Amazon ECR image access
- CloudWatch logging
- X-Ray tracing
- AgentCore workload identity access

### Additional Permissions
- **None required** - AWS documentation is publicly accessible

The role is automatically configured with the proper trust policy for bedrock-agentcore.amazonaws.com service.

In [6]:
tool_name = "aws_doc_mcp_server_ibc"
additional_managed_policies = []  # No additional permissions needed for public documentation
print(f"Creating IAM role for {tool_name}...")
agentcore_iam_role = create_agentcore_role(
    agent_name=tool_name, 
    managed_policies=additional_managed_policies
)
print(f"IAM role created ✓")
print(f"Role ARN: {agentcore_iam_role['Role']['Arn']}")

Creating IAM role for aws_doc_mcp_server_ibc...
attaching inline role policy agentcore-aws_doc_mcp_server_ibc-role
IAM role created ✓
Role ARN: arn:aws:iam::241533163649:role/agentcore-aws_doc_mcp_server_ibc-role


## 5. AgentCore Runtime Configuration

Configure the AgentCore Runtime deployment using the Bedrock AgentCore Starter Toolkit:

### Configuration Parameters
- **Entrypoint**: `mcp-server.py` (our FastMCP wrapper)
- **Execution Role**: The IAM role created above
- **Requirements**: `requirements.txt` with all dependencies
- **Protocol**: MCP (Model Context Protocol)
- **Authentication**: Custom JWT authorizer with Cognito

### Auto-Generated Resources
- Dockerfile optimized for the MCP server
- Amazon ECR repository for container storage
- AgentCore Runtime configuration

The configuration validates that all required files exist before proceeding.

In [7]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp-server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp-server.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name=tool_name
)
print("Configuration completed ✓")

Entrypoint parsed: file=/Users/dohtem/Downloads/claude/ibc2025-acme-corp-bedrockagentcore-chatbot/aws-mcp-server-agentcore/mcp-server.py, bedrock_agentcore_name=mcp-server
Configuring BedrockAgentCore agent: aws_doc_mcp_server_ibc


Using AWS region: eu-central-1
All required files found ✓
Configuring AgentCore Runtime...


Generated Dockerfile: /Users/dohtem/Downloads/claude/ibc2025-acme-corp-bedrockagentcore-chatbot/aws-mcp-server-agentcore/Dockerfile
Generated .dockerignore: /Users/dohtem/Downloads/claude/ibc2025-acme-corp-bedrockagentcore-chatbot/aws-mcp-server-agentcore/.dockerignore
Changing default agent from 'aws_documentation_mcp_server' to 'aws_doc_mcp_server_ibc'
Bedrock AgentCore configured: /Users/dohtem/Downloads/claude/ibc2025-acme-corp-bedrockagentcore-chatbot/aws-mcp-server-agentcore/.bedrock_agentcore.yaml


Configuration completed ✓


## 6. Deployment to AgentCore Runtime

Launch the MCP server to AgentCore Runtime. This process:

### Build and Deploy Steps
1. **Container Build**: Creates Docker image from the generated Dockerfile
2. **ECR Push**: Uploads the container to Amazon ECR
3. **Runtime Creation**: Deploys the AgentCore Runtime
4. **Service Registration**: Registers the MCP server endpoint

### Expected Outputs
- Agent ARN: Unique identifier for the deployed runtime
- Agent ID: Short identifier for management operations
- ECR URI: Container image location

**Note**: This process typically takes 5-10 minutes to complete.

In [8]:
import boto3
import json

iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]
region = boto3.Session().region_name

role_name = 'agentcore-aws_doc_mcp_server_ibc-role'

# Define the correct trust policy for bedrock-agentcore
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AssumeRolePolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock-agentcore.amazonaws.com"
            },
            "Action": "sts:AssumeRole",
            "Condition": {
                "StringEquals": {
                    "aws:SourceAccount": account_id
                },
                "ArnLike": {
                    "aws:SourceArn": f"arn:aws:bedrock-agentcore:{region}:{account_id}:*"
                }
            }
        }
    ]
}

print(f"Updating trust policy for role: {role_name}")
print(f"Account ID: {account_id}")
print(f"Region: {region}")

try:
    # Update the assume role policy
    iam_client.update_assume_role_policy(
        RoleName=role_name,
        PolicyDocument=json.dumps(trust_policy)
    )
    print("✅ Trust policy updated successfully")

    # Verify the update
    role_info = iam_client.get_role(RoleName=role_name)
    print("\n✅ Role verification:")
    print(f"  - Role ARN: {role_info['Role']['Arn']}")
    print(f"  - Trust policy updated: Yes")

    # Wait for propagation
    import time
    print("\n⏳ Waiting 30 seconds for IAM changes to propagate...")
    time.sleep(30)
    print("✅ IAM propagation wait completed")

    print("\n✅ Role is now ready. You can retry the launch operation.")

except Exception as e:
    print(f"❌ Error updating trust policy: {e}")

Updating trust policy for role: agentcore-aws_doc_mcp_server_ibc-role
Account ID: 241533163649
Region: eu-central-1
✅ Trust policy updated successfully

✅ Role verification:
  - Role ARN: arn:aws:iam::241533163649:role/agentcore-aws_doc_mcp_server_ibc-role
  - Trust policy updated: Yes

⏳ Waiting 30 seconds for IAM changes to propagate...
✅ IAM propagation wait completed

✅ Role is now ready. You can retry the launch operation.


In [9]:
import boto3
import json

iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]
region = boto3.Session().region_name

role_name = 'agentcore-aws_doc_mcp_server_ibc-role'

# Define the corrected ECR policy
ecr_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "ECRAuthToken",
            "Effect": "Allow",
            "Action": [
                "ecr:GetAuthorizationToken"
            ],
            "Resource": "*"
        },
        {
            "Sid": "ECRImageAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:BatchGetImage",
                "ecr:GetDownloadUrlForLayer",
                "ecr:BatchCheckLayerAvailability"
            ],
            "Resource": [
                f"arn:aws:ecr:{region}:{account_id}:repository/bedrock-agentcore-aws_documentation_mcp_server",
                f"arn:aws:ecr:{region}:{account_id}:repository/bedrock-agentcore-aws_documentation_mcp_server/*"
            ]
        }
    ]
}

print(f"Adding ECR permissions to role: {role_name}")
print(f"Account ID: {account_id}")
print(f"Region: {region}")

try:
    # Add the ECR policy as an inline policy
    iam_client.put_role_policy(
        RoleName=role_name,
        PolicyName='ECRAccessPolicy',
        PolicyDocument=json.dumps(ecr_policy)
    )
    print("✅ ECR permissions added successfully")

    # List all policies attached to the role
    inline_policies = iam_client.list_role_policies(RoleName=role_name)
    print(f"\n📋 Inline policies attached to role:")
    for policy in inline_policies['PolicyNames']:
        print(f"  - {policy}")

    # Wait for propagation
    import time
    print("\n⏳ Waiting 20 seconds for IAM changes to propagate...")
    time.sleep(20)
    print("✅ IAM propagation wait completed")

    print("\n✅ ECR permissions are now properly configured. You can retry the launch operation.")

except Exception as e:
    print(f"❌ Error adding ECR permissions: {e}")

Adding ECR permissions to role: agentcore-aws_doc_mcp_server_ibc-role
Account ID: 241533163649
Region: eu-central-1
✅ ECR permissions added successfully

📋 Inline policies attached to role:
  - AgentCorePolicy
  - ECRAccessPolicy

⏳ Waiting 20 seconds for IAM changes to propagate...
✅ IAM propagation wait completed

✅ ECR permissions are now properly configured. You can retry the launch operation.


In [10]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'aws_doc_mcp_server_ibc' to account 241533163649 (eu-central-1)
Starting CodeBuild ARM64 deployment for agent 'aws_doc_mcp_server_ibc' to account 241533163649 (eu-central-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: aws_doc_mcp_server_ibc


Launching MCP server to AgentCore Runtime...
This may take several minutes...
Repository doesn't exist, creating new ECR repository: bedrock-agentcore-aws_doc_mcp_server_ibc


✅ ECR repository available: 241533163649.dkr.ecr.eu-central-1.amazonaws.com/bedrock-agentcore-aws_doc_mcp_server_ibc
Using execution role from config: arn:aws:iam::241533163649:role/agentcore-aws_doc_mcp_server_ibc-role
✅ Execution role validation passed: arn:aws:iam::241533163649:role/agentcore-aws_doc_mcp_server_ibc-role
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: aws_doc_mcp_server_ibc
Role name: AmazonBedrockAgentCoreSDKCodeBuild-eu-central-1-3af2d7787d
CodeBuild role doesn't exist, creating new role: AmazonBedrockAgentCoreSDKCodeBuild-eu-central-1-3af2d7787d
Creating IAM role: AmazonBedrockAgentCoreSDKCodeBuild-eu-central-1-3af2d7787d
✓ Role created: arn:aws:iam::241533163649:role/AmazonBedrockAgentCoreSDKCodeBuild-eu-central-1-3af2d7787d
Attaching inline policy: CodeBuildExecutionPolicy to role: AmazonBedrockAgentCoreSDKCodeBuild-eu-central-1-3af2d7787d
✓ Policy attached: CodeBuildExecutionPolicy
Waiting for IAM role

Launch completed ✓
Agent ARN: arn:aws:bedrock-agentcore:eu-central-1:241533163649:runtime/aws_doc_mcp_server_ibc-8F7VmKAEcL
Agent ID: aws_doc_mcp_server_ibc-8F7VmKAEcL


## 7. Runtime Status Monitoring

Monitor the AgentCore Runtime deployment status:

### Status States
- **CREATING**: Runtime is being deployed
- **READY**: Runtime is operational and ready to serve requests
- **CREATE_FAILED**: Deployment failed
- **UPDATE_FAILED**: Update operation failed
- **DELETE_FAILED**: Deletion operation failed

The monitoring loop checks status every 10 seconds until reaching a terminal state. Only proceed to testing when status is **READY**.

In [11]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")

Retrieved Bedrock AgentCore status for: aws_doc_mcp_server_ibc


Initial status: READY
✓ AgentCore Runtime is READY!


## 8. Configuration Storage

Store deployment configuration for remote access:

### AWS Systems Manager Parameter Store
- **Parameter**: `/aws_documentation_mcp_server/runtime/agent_arn`
- **Value**: The Agent ARN from deployment
- **Purpose**: Runtime endpoint identification

### AWS Secrets Manager
- **Secret**: `aws_documentation_mcp_server/cognito/credentials`
- **Content**: Complete Cognito configuration including bearer token
- **Purpose**: Authentication for remote client access

This configuration enables the remote client to authenticate and connect to the deployed MCP server.

In [12]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

# Store Cognito credentials in Secrets Manager (this part worked)
try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp/aws_documentation_server-ibc/cognito/credentials',
        Description='Cognito credentials for AWS Documentation MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp/aws_documentation_server-ibc/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

# Use a different parameter name that doesn't start with "aws"
agent_arn_response = ssm_client.put_parameter(
    Name='/mcp/documentation_server-ibc/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for AWS Documentation MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Parameter path: /mcp/documentation_server-ibc/runtime/agent_arn")
print(f"Secret path: mcp/aws_documentation_server-ibc/cognito/credentials")



✓ Cognito credentials stored in Secrets Manager
✓ Agent ARN stored in Parameter Store

Configuration stored successfully!
Agent ARN: arn:aws:bedrock-agentcore:eu-central-1:241533163649:runtime/aws_doc_mcp_server_ibc-8F7VmKAEcL
Parameter path: /mcp/documentation_server-ibc/runtime/agent_arn
Secret path: mcp/aws_documentation_server-ibc/cognito/credentials


In [13]:
%%writefile mcp_client_remote.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name

    print(f"Using AWS region: {region}")

    try:
        ssm_client = boto3.client('ssm', region_name=region)
        # Updated parameter path - doesn't start with "aws"
        agent_arn_response = ssm_client.get_parameter(Name='/mcp/documentation_server-ibc/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        # Try both possible secret paths
        try:
            response = secrets_client.get_secret_value(SecretId='mcp/aws_documentation_server-ibc/cognito/credentials')
        except secrets_client.exceptions.ResourceNotFoundException:
            # Fallback to the original path if the new one doesn't exist
            response = secrets_client.get_secret_value(SecretId='aws_documentation_mcp_server-ibc/cognito/credentials')

        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")

    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)

    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)

    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json",
        "Accept": "application/json, text/event-stream"
    }

    print(f"\nConnecting to: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()

                tool_result = await session.list_tools()

                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"📚 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()

                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} AWS Documentation tools available.")

    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

Overwriting mcp_client_remote.py


## 9. Remote Testing

Test the deployed MCP server using the remote client (`mcp_client_remote.py`):

### Remote Client Features
- **Credential Retrieval**: Automatically fetches Agent ARN and bearer token from AWS
- **HTTPS Connection**: Connects to the AgentCore Runtime endpoint
- **Tool Discovery**: Lists all available AWS Documentation tools
- **Error Handling**: Provides detailed error messages for troubleshooting

### Expected Output
The client should display AWS Documentation tools with their descriptions and parameters:
- **Global AWS** (3 tools): read_documentation, search_documentation, recommend
- **AWS China** (2 tools): read_documentation, get_available_services

### Connection Details
- **Endpoint**: `https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations`
- **Authentication**: Bearer token from Cognito
- **Protocol**: MCP over HTTPS

In [14]:
print("Testing deployed MCP server...")
print("=" * 50)
!source .venv/bin/activate && python3 mcp_client_remote.py

Testing deployed MCP server...
Using AWS region: eu-central-1
Retrieved Agent ARN: arn:aws:bedrock-agentcore:eu-central-1:241533163649:runtime/aws_doc_mcp_server_ibc-8F7VmKAEcL
✓ Retrieved bearer token from Secrets Manager

Connecting to: https://bedrock-agentcore.eu-central-1.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aeu-central-1%3A241533163649%3Aruntime%2Faws_doc_mcp_server_ibc-8F7VmKAEcL/invocations?qualifier=DEFAULT

🔄 Initializing MCP session...

📋 Available MCP Tools:
📚 read_documentation
   Description: Fetch and convert an AWS documentation page to markdown format.

    ## Usage

    This tool retrieves the content of an AWS documentation page and converts it to markdown format.
    For long documents, you can make multiple calls with different start_index values to retrieve
    the entire content in chunks.

    ## URL Requirements

    - Must be from the docs.aws.amazon.com domain
    - Must end with .html

    ## Example URLs

    - https://docs.aws.amazon.com/A

## 10. Resource Cleanup

Clean up all AWS resources created during this demonstration:

### Cleanup Operations
1. **Local Configuration**: Remove `.bedrock_agentcore.yaml`
2. **AgentCore Runtime**: Delete the deployed runtime
3. **ECR Repository**: Remove container images and repository
4. **IAM Role**: Detach policies and delete execution role
5. **Parameter Store**: Remove stored Agent ARN
6. **Secrets Manager**: Delete Cognito credentials

### Important Notes
- Cleanup is irreversible - ensure you no longer need the deployed resources
- Some resources may have dependencies that prevent immediate deletion
- Manual cleanup may be required if automated cleanup fails

**Warning**: This will permanently delete your deployed MCP server and all associated resources.

In [ ]:
import boto3

print("🗑️  Starting cleanup process...")

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)
iam_client = boto3.client('iam')
ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    print("Clean up agentcore-config file")
    config_file = ".bedrock_agentcore.yaml"
    if os.path.exists(config_file):
        os.remove(config_file)
        print(f"✅ Successfully removed {config_file}")

    print("Deleting AgentCore Runtime...")
    runtime_delete_response = agentcore_control_client.delete_agent_runtime(
        agentRuntimeId=launch_result.agent_id,
    )
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository...")
    ecr_repo_name = launch_result.ecr_uri.split('/')[1]
    ecr_client.delete_repository(
        repositoryName=ecr_repo_name,
        force=True
    )
    print("✓ ECR repository deleted")

    print("Deleting IAM role policies...")
    policies = iam_client.list_role_policies(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        MaxItems=100
    )

    for policy_name in policies['PolicyNames']:
        iam_client.delete_role_policy(
            RoleName=agentcore_iam_role['Role']['RoleName'],
            PolicyName=policy_name
        )
    
    # List attached managed policies
    policies = iam_client.list_attached_role_policies(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        MaxItems=100
        )

    for policy in policies['AttachedPolicies']:
        policy_arn = policy['PolicyArn']
        iam_client.detach_role_policy(
            RoleName=agentcore_iam_role['Role']['RoleName'],
            PolicyArn=policy_arn
        )

    iam_client.delete_role(
        RoleName=agentcore_iam_role['Role']['RoleName']
    )
    print("✓ IAM role deleted")

    try:
        ssm_client.delete_parameter(Name='/aws_documentation_mcp_server/runtime/agent_arn')
        print("✓ Parameter Store parameter deleted")
    except ssm_client.exceptions.ParameterNotFound:
        print("ℹ️  Parameter Store parameter not found")

    try:
        secrets_client.delete_secret(
            SecretId='aws_documentation_mcp_server/cognito/credentials',
            ForceDeleteWithoutRecovery=True
        )
        print("✓ Secrets Manager secret deleted")
    except secrets_client.exceptions.ResourceNotFoundException:
        print("ℹ️  Secrets Manager secret not found")

    print("\n✅ Cleanup completed successfully!")
    
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")